In [ ]:
import pandas as pd
import pyodbc
import numpy as np

def connect_to_db():
    conn = pyodbc.connect('Driver={SQL Server};'
                         'Server=JUANCARLOSRUIZA;'
                         'Database=CzechBankingAnalysis;'
                         'Trusted_Connection=yes;')
    return conn

def import_demograph_data(file_path):
    # Read CSV
    df = pd.read_csv(file_path, sep=';', header=0)
    
    # Display basic info
    print("DataFrame shape:", df.shape)
    print("First few rows:")
    print(df.head())
    print("Data types:")
    print(df.dtypes)
    
    # Rename columns to match your database
    column_names = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 
                   'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16']
    df.columns = column_names
    
    # IMPORTANT: Explicitly convert A3 to string type
    # This is the key change to fix the NULL issue
    df['A3'] = df['A3'].astype(str)
    # Replace 'nan' string with empty string to avoid 'nan' text in database
    df['A3'] = df['A3'].replace('nan', '')
    
    # Convert DataFrame to list of rows for insertion debugging
    print("After string conversion, first few A3 values:")
    print(df['A3'].head().tolist())
    
    # Replace '?' with NaN
    df = df.replace('?', np.nan)
    
    # Convert other columns with numeric data to appropriate types
    # Skip A1 (district_id) and A2 (name) and A3 (region), which are already correct
    for col in column_names[3:]:  # Start from A4
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, invalid values become NaN
    
    # Connect to database and drop existing table
    conn = connect_to_db()
    cursor = conn.cursor()
    
    # Check if table exists and drop it if it does
    cursor.execute("""
    IF OBJECT_ID('dbo.Demograph', 'U') IS NOT NULL
        DROP TABLE dbo.Demograph
    """)
    conn.commit()
    
    # Create table with appropriate types 
    cursor.execute("""
    CREATE TABLE dbo.Demograph (
        A1 INT PRIMARY KEY,
        A2 NVARCHAR(100),
        A3 NVARCHAR(100),    -- Ensure this is NVARCHAR, not INT
        A4 FLOAT,
        A5 FLOAT,
        A6 INT,
        A7 INT,
        A8 INT,
        A9 INT,
        A10 FLOAT,
        A11 INT,
        A12 FLOAT,
        A13 FLOAT,
        A14 INT,
        A15 INT,
        A16 INT
    )
    """)
    conn.commit()
    
    # Insert data row by row
    for index, row in df.iterrows():
        # Print debugging info for the first few rows
        if index < 5:
            print(f"Row {index} values: {row.tolist()}")
            
        # Create a parameterized query
        query = """
        INSERT INTO Demograph 
        (A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, A14, A15, A16)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        
        # Extract values and handle NaN values
        values = []
        for i, col in enumerate(column_names):
            val = row[col]
            if pd.isna(val):
                values.append(None)
            else:
                # Special handling for A3 to ensure it's a string
                if col == 'A3' and val == 'nan':
                    values.append('')  # Replace 'nan' with empty string
                else:
                    values.append(val)
        
        # Execute with explicit values
        cursor.execute(query, values)
        
        # Print confirmation
        if index < 5:
            print(f"Inserted row {index}")
    
    conn.commit()
    conn.close()
    print(f"Successfully imported {len(df)} demograph records")

# Call the function with the path to your CSV file
file_path = 'C:\\Users\\carlo\\Documents\\4.DS\\CAT3.CustomerLVS\\data\\Demograph.csv'
import_demograph_data(file_path)